## Libraries

In [ ]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [ ]:
if IN_COLAB:
    !pip install transformers
    !pip install datasets
    !pip install evaluate
    !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.1 MB/s 
     |████████████████████████████████| 7.6 MB 74.8 MB/s 
     |████████████████████████████████| 182 kB 98.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.0 MB/s 
     |████████████████████████████████| 132 kB 69.1 MB/s 
     |████████████████████████████████| 212 kB 74.1 MB/s 
     |████████████████████████████████| 127 kB 85.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 3.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [ ]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA'
else:
    root_path = '/Users/kscaria/Library/CloudStorage/GoogleDrive-scariakevin1@gmail.com/My Drive/Knowledge/MSIT/Research/InstructABSA'
    
use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.utils import T5ATE
from InstructABSA.data_prep import ModelReadyData

In [ ]:
rest_train_file_path = './Data/semeval14/ABSA_TrainData/Restaurants_Train_v2.csv'
laptops_train_file_path = './Data/semeval14/ABSA_TrainData/Laptop_Train_v2.csv'
rest_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Restaurants_Test_Gold.csv'
laptops_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Laptops_Test_Gold.csv'

# Load the data
res_tr_df = pd.read_csv(rest_train_file_path)
lap_tr_df = pd.read_csv(laptops_train_file_path)
res_te_df = pd.read_csv(rest_test_file_path)
lap_te_df = pd.read_csv(laptops_test_file_path)

In [ ]:
# Extract the aspect term and polarity for each review
model_data = ModelReadyData()

# Get the input text into the required format
bos_instruction_lap = """Definition: The output will be the aspects (both implicit and explicit) which have an associated opinion that are extracted from the input text. In cases where there are no aspects the output should be noaspectterm.\n 
Positive example 1- input: I charge it at night and skip taking the cord with me because of the good battery life. Output: battery life\n
Positive example 2- input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!. Output: features, iChat, Photobooth, garage band\n
Now complete the following example-
input: """

bos_instruction_res = """Definition: The output will be the aspects (both implicit and explicit) which have an associated opinion that are extracted from the input text. In cases where there are no aspects the output should be noaspectterm.\n 
Positive example 1- input: With the great variety on the menu , I eat here often and never get bored. Aspect: menu\n  
Positive example 2- input: Great food, good size menu, great service and an unpretensious setting. Aspect: food, service, ambience\n  
Now complete the following example-
input: """

eos_instruction = ' Output:'
res_tr_df = model_data.create_data_in_ate_format(res_tr_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction_res, eos_instruction)
lap_tr_df = model_data.create_data_in_ate_format(lap_tr_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction_lap, eos_instruction)
res_te_df = model_data.create_data_in_ate_format(res_te_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction_res, eos_instruction)
lap_te_df = model_data.create_data_in_ate_format(lap_te_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction_lap, eos_instruction)

## 10% Training Data

In [ ]:
# Experimentation
experiment_id = 'combined100'
experiment_name = 'combined100_instruct'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'

# Create T5 utils object
t5_exp = T5ATE(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df, sample_size=0.1)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATE')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
dataset, tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495M [00:00<?, ?B/s]

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
# Train Model
model_trainer = t5_exp.train(tokenized_dataset, **training_args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 609
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 156
  Number of trainable parameters = 247534848


Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.451614
2,No log,0.482934
3,No log,0.440686
4,No log,0.444059


***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-p

In [ ]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'validation')

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 609
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints


***** Running Prediction *****
  Num examples = 1600
  Batch size = 16


In [ ]:
# Add new column in the respective dataframes
tr_df = pd.DataFrame(dataset['train'])
te_df = pd.DataFrame(dataset['validation'])

tr_df['pred_labels'] =  tr_pred_labels
te_df['pred_labels'] =  te_pred_labels

# Metrics
print('Train accuracy: ', tr_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(tr_df))
print('Test accuracy: ', te_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
tr_df.to_csv(os.path.join(dump_path, '10pc_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
te_df.to_csv(os.path.join(dump_path, '10pc_' + id_te_filename), index = False)

Train accuracy:  0.9047619047619048
Test accuracy:  0.718125


## 20% Training Data

In [ ]:
# Experimentation
experiment_id = 'combined100'
experiment_name = 'combined100_instruct'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'

# Create T5 utils object
t5_exp = T5ATE(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df, sample_size=0.2)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATE')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
dataset, tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
# Train Model
model_trainer = t5_exp.train(tokenized_dataset, **training_args)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1217
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 308
  Number of trainable parameters = 247534848
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.450803
2,No log,0.353048
3,No log,0.335077
4,No log,0.328742


***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-p

In [ ]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'validation')

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1217
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints


***** Running Prediction *****
  Num examples = 1600
  Batch size = 16


In [ ]:
# Add new column in the respective dataframes
tr_df = pd.DataFrame(dataset['train'])
te_df = pd.DataFrame(dataset['validation'])

tr_df['pred_labels'] =  tr_pred_labels
te_df['pred_labels'] =  te_pred_labels

# Metrics
print('Train accuracy: ', tr_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(tr_df))
print('Test accuracy: ', te_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
tr_df.to_csv(os.path.join(dump_path, '20pc_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
te_df.to_csv(os.path.join(dump_path, '20pc_' + id_te_filename), index = False)

Train accuracy:  0.8241577649958916
Test accuracy:  0.740625


## 50% Training Data

In [ ]:
device = cuda.get_current_device()
device.reset()

In [ ]:
# Experimentation
experiment_id = 'combined100'
experiment_name = 'combined100_instruct'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'

# Create T5 utils object
t5_exp = T5ATE(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df, sample_size=0.5)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATE')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
dataset, tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
# Train Model
model_trainer = t5_exp.train(tokenized_dataset, **training_args)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3043
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 764
  Number of trainable parameters = 247534848
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.329994
2,No log,0.257303
3,0.542300,0.242984
4,0.542300,0.240780


***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-p

In [ ]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'validation')

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3043
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints


***** Running Prediction *****
  Num examples = 1600
  Batch size = 16


In [ ]:
# Add new column in the respective dataframes
tr_df = pd.DataFrame(dataset['train'])
te_df = pd.DataFrame(dataset['validation'])

tr_df['pred_labels'] =  tr_pred_labels
te_df['pred_labels'] =  te_pred_labels

# Metrics
print('Train accuracy: ', tr_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(tr_df))
print('Test accuracy: ', te_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
tr_df.to_csv(os.path.join(dump_path, '50pc_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
te_df.to_csv(os.path.join(dump_path, '50pc_' + id_te_filename), index = False)

Train accuracy:  0.8642786723627999
Test accuracy:  0.82375
